# Metadata extraction using DSPy and a local LLM using GEPA optimization

To run this, you first need to start two local vLLM servers in the backround. These commands are tested on a single A100 80GB in non-exclusive mode (e.g. Turso oversub GPU). The GPT-OSS 120B model has to be partially offloaded to CPU to preserve VRAM.

For the main extractor model:

    vllm serve google/gemma-3-4b-it --port 7987 --max-model-len 16384 --gpu-memory-utilization 0.25

For the reflection model:
    
    llama-server -hf ggml-org/gpt-oss-120b-GGUF --host 0.0.0.0 --port 7988 --ctx-size 0 --jinja -ub 2048 -b 2048 --n-cpu-moe 5


In [1]:
import dspy

MODEL_ID = "google/gemma-3-4b-it"  # should match the model vLLM is running (does it matter??)
PORT = 7987  # should match the port where vLLM is running
MAX_TOKENS = 2048  # limit on how many new tokens to generate (default: 4000)
TEMPERATURE = 0.7

lm = dspy.LM("openai/" + MODEL_ID,
             api_base=f"http://localhost:{PORT}/v1",  # ensure this points to your port
             api_key="local", model_type="chat", max_tokens=MAX_TOKENS, temperature=TEMPERATURE)
dspy.configure(lm=lm)

# test the connection to the LLM
lm("Who are you?", temperature=0.0)

["I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m widely available for public use! \n\nI can take text and images as inputs and generate text-based responses. \n\nYou can learn more about me on the Gemma project page: [https://ai.google.dev/gemma](https://ai.google.dev/gemma)"]

In [2]:

REFLECTION_MODEL_ID = "ggml-org/gpt-oss-120b-GGUF"
REFLECTION_PORT = PORT + 1
REFLECTION_MAX_TOKENS = 16384

reflection_lm = dspy.LM("openai/" + REFLECTION_MODEL_ID,
             api_base=f"http://localhost:{REFLECTION_PORT}/v1",  # ensure this points to your port
             api_key="local", model_type="chat", max_tokens=REFLECTION_MAX_TOKENS, temperature=TEMPERATURE)

# test the connection to the LLM
reflection_lm("Who are you?", temperature=0.0)

['I’m ChatGPT\u202f—\u202fa large language model created by OpenAI. I’ve been trained on a wide variety of text up through June\u202f2024, which lets me help with things like answering questions, brainstorming ideas, explaining concepts, drafting or editing writing, solving problems, and much more. I don’t have personal experiences or consciousness, and I can’t browse the web in real time, but I can draw on the information I was trained on to generate useful, context‑aware responses.  \n\nIf there’s anything specific you’d like to know or discuss, just let me know!']

In [3]:
# Load and prepare dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of validation set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

VAL_SIZE = 64  # how many documents to validate on during optimization

def preprocess_sample(sample):
    # fix some bad field names
    ground_truth = { fld.replace('-', '_'): val for fld, val in sample["ground_truth"].items() }
    output = json.dumps(ground_truth)
    input_ = json.dumps(sample["content"])
    return dspy.Example({"content": input_, "metadata": output}).with_inputs("content")

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records


train_val_set = dataset_to_records(train_files)
random.shuffle(train_val_set)

train_set = train_val_set[VAL_SIZE:]
val_set = train_val_set[:VAL_SIZE]

test_set = dataset_to_records(test_files)

len(train_set), len(val_set), len(test_set)

(576, 64, 182)

In [4]:
print("Input Message:")
print(train_set[-1]['content'])

print("\n\nGold Answer:")
for k, v in json.loads(train_set[-1]['metadata']).items():
    print(f"{k}: {v}")

Input Message:
{"pdfinfo": {"creationDate": "D:20201214215341+01'00'", "modDate": "D:20201214215418+01'00'"}, "pages": [{"page": 1, "text": "# ANTAA TAITEEN OPETTAA\n\n\n"}, {"page": 3, "text": "ANTA A TAITEEN OPETTA A GERT BIESTA\n\n\n"}, {"page": 4, "text": "00:00:08.18\n\n\n"}, {"page": 5, "text": "00:00:36.03 00:00:52.19 00:00:54.19\n\n\n"}, {"page": 6, "text": "00:00:58.16 00:01:00.17 00:01:0\n\n\n"}, {"page": 65, "text": "\u2018Opastan sinua kaikessa, n\u00e4yt\u00e4n sinulle kaiken ja nime\u00e4n kaiken.\u2019\n\u2014 COMENIUS\nT\u00e4ss\u00e4 kirjassa Gert Biesta esitt\u00e4\u00e4 uuden n\u00e4kemyksen nykyaikaisesta taidekasvatuksesta\n\nosoittamalla, ett\u00e4 taide tarjoaa ainutlaatuisia v\u00e4lineit\u00e4 olla dialogissa maailman kanssa. N\u00e4kemys\n\nperustuu ajatukseen, ett\u00e4 opettaminen on n\u00e4ytt\u00e4mist\u00e4. Opettaja n\u00e4ytt\u00e4\u00e4 oppilaalle millaisiin\n\nhyviin, t\u00e4rkeisiin tai merkitt\u00e4viin asioihin maailmassa voisi kiinnitt\u00e4\u00e4

In [5]:
from typing import Optional

class ExtractInfo(dspy.Signature):
    """Extract structured metadata from text extracted from a PDF."""

    content: str = dspy.InputField()
    language: str = dspy.OutputField(desc="The language of the resource expressed as a BCP47 language tag.")
    title: str = dspy.OutputField(desc="The main title of the publication.")
    alt_title: list[str] = dspy.OutputField(desc="Alternative or parallel titles of the publication, suffixed with a BCP47 language tag in curly brackets.")
    creator: list[str] = dspy.OutputField(desc="The primary author(s) of the resource (order: Last Name, First Names).")
    year: Optional[str] = dspy.OutputField(desc="The year on which the resource was issued or made available.")
    publisher: list[str] = dspy.OutputField(desc="The entity/entities responsible for making the resource available.")
    doi: Optional[str] = dspy.OutputField(desc="The Digital Object Identifier (DOI) associated with the resource.")
    e_isbn: list[str] = dspy.OutputField(desc="The ISBN associated with the electronic resource.")
    p_isbn: list[str] = dspy.OutputField(desc="The ISBN of the printed version of this document.")
    e_issn: Optional[str] = dspy.OutputField(desc="The ISSN associated with the electronic resource.")
    p_issn: Optional[str] = dspy.OutputField(desc="The ISSN of the printed version of this document.")
    type_coar: str = dspy.OutputField(desc="The type of the resource according to the COAR Resource Types classification.")

module = dspy.ChainOfThought(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(content=text)

print(response)


Prediction(
    reasoning='The text describes an announcement by Apple Inc. regarding the iPhone 14. It mentions the CEO, Tim Cook, and a press release. I will extract the key entities and information to populate the metadata fields.',
    language='en',
    title='iPhone 14 Announcement',
    alt_title=[],
    creator=['Apple Inc.', 'Tim Cook'],
    year=None,
    publisher=['Apple Inc.'],
    doi=None,
    e_isbn=[],
    p_isbn=[],
    e_issn=None,
    p_issn=None,
    type_coar='News Article'
)


In [6]:
import Levenshtein

ALMOST_THRESHOLD = 0.95  # Adjust as needed

def feedback_simple_string(field, true_val, pred_val):
    score = 1.0 if true_val == pred_val else 0.0
    if score == 1.0:
        feedback = f"✅ `{field}` is correct: `{true_val}`."
    else:
        feedback = f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."
    return score, feedback

def feedback_fuzzy_string(field, true_val, pred_val):
    base_score = 1.0 if true_val == pred_val else 0.0
    if base_score == 1.0 or (true_val and pred_val and Levenshtein.ratio(true_val.lower(), pred_val.lower()) >= ALMOST_THRESHOLD):
        score = 1.0
        feedback = f"✅ `{field}` is approximately correct: `{pred_val}` matches `{true_val}` closely."
    else:
        score = 0.0
        feedback = f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."
    return score, feedback

def feedback_set(field, true_val, pred_val):
    true_set = set(true_val or [])
    pred_set = set(pred_val or [])

    if not true_set and not pred_set:
        return 1.0, f"✅ `{field}` is empty as expected."
    elif not true_set or not pred_set:
        return 0.0, f"❌ `{field}` is incorrect. Expected `{true_set}`, but got `{pred_set}`."

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    feedback = f"🔍 `{field}` partial match."
    feedback += f"- Correctly included: `{list(true_set & pred_set)}`\n"
    if fp:
        feedback += f"- Incorrectly included: `{list(pred_set - true_set)}`\n"
    if fn:
        feedback += f"- Missed: `{list(true_set - pred_set)}`"

    return f1, feedback.strip()

def feedback_e_issn(field, true_val, pred_val, p_issn_val):
    if true_val == pred_val:
        return 1.0, f"✅ `{field}` is correct: `{true_val}`."
    elif p_issn_val and pred_val == p_issn_val and true_val is None:
        return 1.0, f"✅ `{field}` is correctly inferred from `p_issn`: `{pred_val}`."
    else:
        return 0.0, f"❌ `{field}` is incorrect. You predicted `{pred_val}`, but the correct value is `{true_val}`."

def metadata_metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    fields = [
        'language', 'title', 'creator', 'year', 'publisher',
        'doi', 'e_isbn', 'p_isbn', 'e_issn', 'p_issn', 'type_coar'
    ]

    scores = []
    feedback_parts = []

    metadata = json.loads(example.get("metadata", "{}"))
    ground_truth = example.get("ground_truth", {})

    for field in fields:
        true_val = metadata.get(field)
        pred_val = pred.get(field) or None

        if field in ['language', 'year', 'doi', 'p_issn', 'type_coar']:
            score, feedback = feedback_simple_string(field, true_val, pred_val)
        elif field == 'title':
            score, feedback = feedback_fuzzy_string(field, true_val, pred_val)
        elif field in ['creator', 'publisher', 'e_isbn', 'p_isbn']:
            score, feedback = feedback_set(field, true_val, pred_val)
        elif field == 'e_issn':
            p_issn_val = ground_truth.get("p_issn")
            score, feedback = feedback_e_issn(field, true_val, pred_val, p_issn_val)
        else:
            score, feedback = feedback_simple_string(field, true_val, pred_val)

        scores.append(score)
        feedback_parts.append(feedback)

    overall_score = sum(scores) / len(scores) if scores else 0
    full_feedback = "\n".join(feedback_parts)

    return dspy.Prediction(score=overall_score, feedback=full_feedback)


In [7]:
from dspy import GEPA

optimizer = GEPA(
    metric=metadata_metric_with_feedback,
#    auto="heavy",
    max_metric_calls=3200,
    num_threads=64,
    track_stats=False,
    use_merge=True,
    reflection_lm=reflection_lm
)

In [8]:
%%time

optimized_program = optimizer.compile(
    module,
    trainset=train_set,
    valset=val_set,
)

2025/09/30 15:01:06 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 3200 metric calls of the program. This amounts to 5.00 full evals on the train+val set.
2025/09/30 15:01:06 INFO dspy.teleprompt.gepa.gepa: Using 64 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/3200 [00:00<?, ?rollouts/s]2025/09/30 15:01:07 INFO dspy.evaluate.evaluate: Average Metric: 37.57647907647908 / 64 (58.7%)
2025/09/30 15:01:07 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.5871324855699855
GEPA Optimization:   2%|▏         | 64/3200 [00:00<00:34, 90.29rollouts/s]2025/09/30 15:01:07 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.5871324855699855


Average Metric: 1.97 / 3 (65.7%): 100%|██████████| 3/3 [00:00<00:00, 93.72it/s]

2025/09/30 15:01:07 INFO dspy.evaluate.evaluate: Average Metric: 1.9696969696969697 / 3 (65.7%)


2025/09/30 15:02:21 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: **Task Overview**
You will receive a JSON object that contains the result of extracting raw text from a PDF file (`pdfinfo` + a list of pages with their text).  
Your job is to *automatically* parse this information and produce a structured metadata record in JSON‑like key/value form (plain text, not a JSON object).  

**Required output fields (exact names and order)**
```
reasoning
language
title
alt_title
creator
year
publisher
doi
e_isbn
p_isbn
e_issn
p_issn
type_coar
```
Each field must be on its own line, exactly as shown above, followed by a colon, a space, and the value.  
Values must follow the formats described below. Do **not** add any extra fields, headings, or markup.

---

### 1. General parsing strategy (you may follow this step‑by‑step)

1. **Read `pdfinfo`** – use the `title`, `author`, `creationDate`, and `modDate` fields as fall‑backs only when the page text does not contai

Average Metric: 1.73 / 3 (57.6%): 100%|██████████| 3/3 [00:00<00:00, 103.32it/s]

...

2025/09/30 18:04:54 INFO dspy.teleprompt.gepa.gepa: Iteration 78: Proposed new text for predict: markdown
# Instruction for extracting structured metadata from PDF‑extracted text

You will receive **one JSON object** with two top‑level keys:

* **pdfinfo** – optional dictionary containing PDF‑level metadata (`title`, `author`, `creationDate`, `modDate`, …).
* **pages** – a list of page objects, each with an integer `page` number and a `text` string that is the raw OCR/clipboard text of that page.

Your job is to **scan the whole document** (all pages) and produce a **flat list of metadata fields**.  
Write each field name on its own line, followed by the extracted value on the next line (see *Output format*).  
If a value cannot be found, output the placeholder described in the *Missing‑value handling* table.

---

## 1. Required output fields

| Field name | Type | Notes |
|------------|------|-------|
| **language** | string | ISO‑639‑1 code of the document language: `fi`, `en`, `sv`

Average Metric: 1.61 / 3 (53.5%): 100%|██████████| 3/3 [00:05<00:00,  1.91s/it]

2025/09/30 18:05:42 INFO dspy.evaluate.evaluate: Average Metric: 1.606060606060606 / 3 (53.5%)


2025/09/30 18:07:14 INFO dspy.teleprompt.gepa.gepa: Iteration 79: Proposed new text for predict: markdown
# Instruction for extracting structured metadata from PDF‑extracted JSON

You will receive **one JSON object** with two top‑level keys:

* **pdfinfo** – optional dictionary containing PDF‑level metadata (`title`, `author`, `creationDate`, `modDate`, …).  
* **pages** – a list of page objects, each with an integer `page` number and a `text` string that is the raw OCR/clipboard text of that page.

Your job is to **scan the whole document** (all pages) and produce a **flat list of metadata fields**.  
Write each field name on its own line, followed by the extracted value on the next line (see *Output format*).  
If a value cannot be found, output the placeholder described in the *Missing‑value handling* table.

---

## 1. Required output fields

| Field name | Type | Notes |
|------------|------|-------|
| **language** | string | ISO‑639‑1 code of the document language: `fi`, `en`, `s

CPU times: user 1min 9s, sys: 17.5 s, total: 1min 27s
Wall time: 3h 6min 51s


In [9]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: predict
Prompt:
markdown
# 📄 Instruction for extracting structured metadata from PDF‑extracted text

You will receive **one JSON object** with the following structure:

* `pdfinfo` – optional dictionary containing PDF‑level metadata (e.g. `author`, `title`, `creationDate`, `modDate`).  
* `pages` – list of page objects, each with a numeric `page` field and a `text` field that holds the raw OCR/clipboard text of that page.

Your job is to **parse this information and output a flat list of metadata fields** (one field name on a line, the value on the next line).  
The output must follow the exact format shown in the “Output format example” section below, because it will later be turned into a JSON record.

---

## 1️⃣ Required output fields (order does **not** matter)

| Field name | Type | Description & format |
|------------|------|----------------------|
| `language` | `string` or `None` | ISO‑639‑1 code of the **content language** (`en`, `fi`, `sv`, `se`, …). Detect from t

In [10]:
%%time

evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metadata_metric_with_feedback,
    num_threads=64,
    display_table=True,
    display_progress=True,
    provide_traceback=True
)

eval_result = evaluate(optimized_program)

Average Metric: 119.53 / 181 (66.0%):  99%|█████████▉| 181/182 [01:57<00:00,  2.07it/s]

2025/09/30 18:10:12 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=2048. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.
2025/09/30 18:10:12 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/30 18:10:35 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=2048. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.
2025/09/30 18:10:35 ERROR dspy.utils.parallelizer: Error for Example({'content': '{"pdfinfo": {"author": "ArtMedia", "creationDate": 

Average Metric: 119.53 / 181 (66.0%): 100%|██████████| 182/182 [02:36<00:00,  1.16it/s]

2025/09/30 18:10:35 INFO dspy.evaluate.evaluate: Average Metric: 119.52768595041326 / 182 (65.7%)
2025/09/30 18:10:35 WARNING dspy.evaluate.evaluate: Skipping table display since `pandas` is not installed.



CPU times: user 4.28 s, sys: 844 ms, total: 5.13 s
Wall time: 2min 36s


In [11]:
lm.inspect_history()





[2025-09-30T18:10:35.498812]

System message:

Your input fields are:
1. `content` (str):
Your output fields are:
1. `reasoning` (str): 
2. `language` (str): The language of the resource expressed as a BCP47 language tag.
3. `title` (str): The main title of the publication.
4. `alt_title` (list[str]): Alternative or parallel titles of the publication, suffixed with a BCP47 language tag in curly brackets.
5. `creator` (list[str]): The primary author(s) of the resource (order: Last Name, First Names).
6. `year` (Union[str, NoneType]): The year on which the resource was issued or made available.
7. `publisher` (list[str]): The entity/entities responsible for making the resource available.
8. `doi` (Union[str, NoneType]): The Digital Object Identifier (DOI) associated with the resource.
9. `e_isbn` (list[str]): The ISBN associated with the electronic resource.
10. `p_isbn` (list[str]): The ISBN of the printed version of this document.
11. `e_issn` (Union[str, NoneType]): The ISSN assoc

In [12]:
# save the optimized program for later use (many formats, just in case)
optimized_program.save("gepa-optimized-module.json", save_program=False)
optimized_program.save("gepa-optimized-module.pkl", save_program=False)
# save just the prompt(s)
for name, pred in optimized_program.named_predictors():
    with open(f"gepa-optimized-prompt-{name}.txt", "w") as outfile:
        outfile.write(pred.signature.instructions)
